follows http://www.gaia-gis.it/gaia-sins/spatialite-tutorial-2.3.1.html

In [1]:
import ArchGDAL; const AG = ArchGDAL

ArchGDAL

In [20]:
AG.registerdrivers() do
    AG.read("../test/spatialite/test-2.3.sqlite") do dataset
        print(dataset)
    end
end;

GDAL Dataset (Driver: SQLite/SQLite / Spatialite)
File(s): ../test/spatialite/test-2.3.sqlite 
Number of feature layers: 3
  Layer 0: Towns (wkbPoint), nfeatures = 8101
  Layer 1: HighWays (wkbLineString), nfeatures = 775
  Layer 2: Regions (wkbMultiPolygon), nfeatures = 109


In [7]:
function inspect(query, filename="../test/spatialite/test-2.3.sqlite")
    AG.registerdrivers() do
        AG.read(filename) do dataset
            AG.executesql(dataset, query) do results
                print(results)
            end
        end
    end
end

inspect (generic function with 3 methods)

In [8]:
inspect("SELECT * FROM towns LIMIT 5")

Layer: SELECT, nfeatures = 5
  Geometry 0 (Geometry): [wkbUnknown], POINT (427002.77 499...), ...
     Field 0 (PK_UID): [OFTInteger], 1, 2, 3, 4, 5
     Field 1 (Name): [OFTString], Brozolo, Campiglione-Fenile, Canischio, ...
     Field 2 (Peoples): [OFTInteger], 435, 1284, 274, 2281, 1674
     Field 3 (LocalCounc): [OFTInteger], 1, 1, 1, 1, 1
     Field 4 (County): [OFTInteger], 0, 0, 0, 0, 0
...
 Number of Fields: 6

In [9]:
inspect("""select name, peoples from towns
           WHERE peoples > 350000 order by peoples DESC""")

Layer: SELECT, nfeatures = 8
     Field 0 (Name): [OFTString], Roma, Milano, Napoli, Torino, Palermo, ...
     Field 1 (Peoples): [OFTInteger], 2546804, 1256211, 1004500, 865263, ...


false

In [10]:
inspect("""
select COUNT(*) as '# Towns',
    MIN(peoples) as Smaller,
    MAX(peoples) as Bigger,
    SUM(peoples) as 'Total peoples',
    SUM(peoples) / COUNT(*) as 'mean peoples for town'
from towns
""")

Layer: SELECT, nfeatures = 1
     Field 0 (# Towns): [OFTInteger], 8101
     Field 1 (Smaller): [OFTInteger], 33
     Field 2 (Bigger): [OFTInteger], 2546804
     Field 3 (Total peoples): [OFTInteger], 57006147
     Field 4 (mean peoples for town): [OFTInteger], 7036


false

In [11]:
inspect("""
select (10 - 11) * 2 as Number, ABS((10 - 11) * 2) as AbsoluteValue
""")

Layer: SELECT, nfeatures = 1
     Field 0 (Number): [OFTInteger], -2
     Field 1 (AbsoluteValue): [OFTInteger], 2


false

In [12]:
inspect("""
select name, peoples, HEX(Geometry)
from Towns where peoples > 350000 order by peoples DESC
""")

Layer: SELECT, nfeatures = 8
     Field 0 (Name): [OFTString], Roma, Milano, Napoli, Torino, Palermo, ...
     Field 1 (Peoples): [OFTInteger], 2546804, 1256211, 1004500, 865263, ...
     Field 2 (HEX(Geometry)): [OFTString], 0001787F00003D0AD723..., ...


false

In [13]:
inspect("""
SELECT name, peoples, AsText(Geometry)
from Towns where peoples > 350000 order by peoples DESC
""")

Layer: SELECT, nfeatures = 8
  Geometry 0 (AsText(Geometry)): [wkbUnknown], POINT (788703.57 464...), ...
     Field 0 (Name): [OFTString], Roma, Milano, Napoli, Torino, Palermo, ...
     Field 1 (Peoples): [OFTInteger], 2546804, 1256211, 1004500, 865263, ...


false

In [14]:
inspect("""
SELECT name, X(Geometry), Y(Geometry) FROM Towns
WHERE peoples > 350000 
ORDER BY peoples DESC
""")

Layer: SELECT, nfeatures = 8
     Field 0 (Name): [OFTString], Roma, Milano, Napoli, Torino, Palermo, ...
     Field 1 (X(Geometry)): [OFTReal], 788703.57, 514820.49, 942636.1, ...
     Field 2 (Y(Geometry)): [OFTReal], 4.6456363e6, 5.03453456e6, ...


false

In [15]:
inspect("SELECT HEX(GeomFromText('POINT(10 20)'))")

Layer: SELECT, nfeatures = 1
     Field 0 (HEX(GeomFromText('POINT(10 20)'))): [OFTString], ...


false

In [16]:
inspect("SELECT HEX(AsBinary(GeomFromText('POINT(10 20)')))")

Layer: SELECT, nfeatures = 1
     Field 0 (HEX(AsBinary(GeomFromText('POINT(10 20)')))): [OFTString], ...


false

In [17]:
inspect("SELECT AsText(GeomFromWKB(X'010100000000000000000024400000000000003440'))")

Layer: SELECT, nfeatures = 1
  Geometry 0 (AsText(GeomFromWKB(X'010100000000000000000024400000000000003440'))): [wkbUnknown][1:70]...


false

In [27]:
inspect("SELECT PK_UID, AsText(Geometry) FROM HighWays WHERE PK_UID = 10")

Layer: SELECT, nfeatures = 1
  Geometry 0 (AsText(Geometry)): [wkbUnknown], LINESTRING (676303.2...)
     Field 0 (PK_UID): [OFTInteger], 10


false

In [28]:
inspect("""
SELECT PK_UID, NumPoints(Geometry), GLength(Geometry),
       Dimension(Geometry), GeometryType(Geometry)
FROM HighWays ORDER BY NumPoints(Geometry) DESC LIMIT 5
""")

Layer: SELECT, nfeatures = 5
     Field 0 (PK_UID): [OFTInteger], 774, 775, 153, 205, 773
     Field 1 (NumPoints(Geometry)): [OFTInteger], 6758, 5120, 4325, 3109, ...
     Field 2 (GLength(Geometry)): [OFTReal], 94997.87213441564, ...
     Field 3 (Dimension(Geometry)): [OFTInteger], 1, 1, 1, 1, 1
     Field 4 (GeometryType(Geometry)): [OFTString], LINESTRING, LINESTRING, ...


false

In [29]:
inspect("""
SELECT PK_UID, NumPoints(Geometry),
       AsText(StartPoint(Geometry)), AsText(EndPoint(Geometry)),
       X(PointN(Geometry, 2)), Y(PointN(Geometry, 2))
FROM HighWays ORDER BY NumPoints(Geometry) DESC LIMIT 5
""")

Layer: SELECT, nfeatures = 5
  Geometry 0 (AsText(StartPoint(Geometry))): [wkbUnknown]
  Geometry 1 (AsText(EndPoint(Geometry))): [wkbUnknown]
     Field 0 (PK_UID): [OFTInteger], 774, 775, 153, 205, 773
     Field 1 (NumPoints(Geometry)): [OFTInteger], 6758, 5120, 4325, 3109, ...
     Field 2 (X(PointN(Geometry, 2))): [OFTReal], 632086.0096648833, ...
     Field 3 (Y(PointN(Geometry, 2))): [OFTReal], 4.835625748753577e6, ...


false

In [30]:
inspect("SELECT name, AsText(Geometry) FROM Regions WHERE PK_UID = 52")

Layer: SELECT, nfeatures = 1
  Geometry 0 (AsText(Geometry)): [wkbUnknown], MULTIPOLYGON (((7618...)
     Field 0 (Name): [OFTString], EMILIA-ROMAGNA


false

In [31]:
inspect("""
SELECT PK_UID, Area(Geometry), AsText(Centroid(Geometry)),
       Dimension(Geometry), GeometryType(Geometry)
FROM Regions ORDER BY Area(Geometry) DESC LIMIT 5
""")

Layer: SELECT, nfeatures = 5
  Geometry 0 (AsText(Centroid(Geometry))): [wkbUnknown], ...
     Field 0 (PK_UID): [OFTInteger], 101, 105, 106, 74, 53
     Field 1 (Area(Geometry)): [OFTReal], 2.5779695636913013e10, ...
     Field 2 (Dimension(Geometry)): [OFTInteger], 2, 2, 2, 2, 2
     Field 3 (GeometryType(Geometry)): [OFTString], MULTIPOLYGON, ...


false

In [32]:
inspect("""
SELECT PK_UID, NumInteriorRings(Geometry),
       NumPoints(ExteriorRing(Geometry)), NumPoints(InteriorRingN(Geometry, 1))
FROM regions ORDER BY NumInteriorRings(Geometry) DESC LIMIT 5
""")

Layer: SELECT, nfeatures = 5
     Field 0 (PK_UID): [OFTInteger], 55, 1, 2, 3, 4
     Field 1 (NumInteriorRings(Geometry)): [OFTInteger], 1, 0, 0, 0, 0
     Field 2 (NumPoints(ExteriorRing(Geometry))): [OFTInteger], 602, 6, 12, ...


LoadError: LoadError: MethodError: `length` has no method matching length(::Void)
while loading In[32], in expression starting on line 1

In [33]:
inspect("""
SELECT AsText(InteriorRingN(Geometry, 1)),
       AsText(PointN(InteriorRingN(Geometry, 1), 4)),
       X(PointN(InteriorRingN(Geometry, 1), 5)),
       Y(PointN(InteriorRingN(Geometry, 1), 5))
FROM Regions WHERE PK_UID = 55
""")

Layer: SELECT, nfeatures = 1
  Geometry 0 (AsText(InteriorRingN(Geometry, 1))): [wkbUnknown]
  Geometry 1 (AsText(PointN(InteriorRingN(Geometry, 1), 4))): [wkbUnknown]
     Field 0 (X(PointN(InteriorRingN(Geometry, 1), 5))): [OFTReal], ...
     Field 1 (Y(PointN(InteriorRingN(Geometry, 1), 5))): [OFTReal], ...


false

In [34]:
inspect("""
SELECT Name, AsText(Envelope(Geometry)) FROM Regions LIMIT 5
""")

Layer: SELECT, nfeatures = 5
  Geometry 0 (AsText(Envelope(Geometry))): [wkbUnknown], ...
     Field 0 (Name): [OFTString], VENETO, VENETO, VENETO, VENETO, LIGURIA


false